# Federate Learing Implementation

1.Load the MNIST dataset (or any other dataset like HAM 10000)

In [3]:
device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'
print(device)

/GPU:0


In [4]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

#Normalized
x_train = x_train / 255.0
x_test = x_test / 255.0

print("Training samples:", x_train.shape[0])
print("Test samples:", x_test.shape[0])
print("Image shape:", x_train.shape[1:])


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training samples: 60000
Test samples: 10000
Image shape: (28, 28)


2.Extract two subsets of 600 data points each (without intersection)

In [11]:
import numpy as np
import torch

def create_client_datasets(x_train, y_train, subset_size, printed=False):
  indices = np.arange(x_train.shape[0])
  np.random.shuffle(indices)

  subset1_indices = indices[:subset_size]
  subset2_indices = indices[subset_size:subset_size*2]

  subset1_x = x_train[subset1_indices]
  subset1_y = y_train[subset1_indices]

  subset2_x = x_train[subset2_indices]
  subset2_y = y_train[subset2_indices]

  subset1_x = subset1_x[..., np.newaxis]
  subset2_x = subset2_x[..., np.newaxis]

  if (printed):
    print("Subset 1 X shape:", subset1_x.shape)
    print("Subset 1 Y shape:", subset1_y.shape)
    print("Subset 2 X shape:", subset2_x.shape)
    print("Subset 2 Y shape:", subset2_y.shape)


  client_datasets = [
      (subset1_x, subset1_y),
      (subset2_x, subset2_y)
  ]

  return client_datasets

client_datasets = create_client_datasets(x_train, y_train, 600, printed=True)

Subset 1 X shape: (600, 28, 28, 1)
Subset 1 Y shape: (600,)
Subset 2 X shape: (600, 28, 28, 1)
Subset 2 Y shape: (600,)


3.Create a simple Convolutional Neural Network (2 convolutional layers and 2 dense layers, for example)

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def create_model():
  model = Sequential([
      Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
      MaxPooling2D((2, 2)),
      Conv2D(64, (3, 3), activation='relu'),
      MaxPooling2D((2, 2)),
      Flatten(),
      Dense(128, activation='relu'),
      Dense(10, activation='softmax')  # 10 classes for MNIST digits
  ])
  return model

4.Create a function average_model_parameters(models: iterable, average_weight): iterable that takes a list of models as an argument and returns the weighted average of the parameters of each model.

In [7]:
from collections.abc import Iterable

def average_model_parameters(models: Iterable, average_weight: list):
    model_params = [list(model.get_weights()) for model in models]

    averaged_params = []
    for params in zip(*model_params):
        weighted_sum = sum(weight * param for weight, param in zip(average_weight, params))
        averaged_params.append(weighted_sum)

    return averaged_params


5.Create a function that updates the parameters of a model from a list of values

In [8]:
def update_model_parameters(model, new_weights):
    model.set_weights(new_weights)


6.Create a script/code/function that reproduces Algorithm 1, considering that both models are on your machine. Use an average_weight=[1/2, 1/2]. Reuse the same setup as in the article (50 examples per local batch)

In [9]:
import numpy as np
from tensorflow.keras.models import clone_model
import tensorflow as tf

def federated_averaging(global_model, client_datasets, num_clients, rounds, epochs, batch_size, learning_rate, loss = 'sparse_categorical_crossentropy', common_parameter=True, fraction_clients=0.5, average_weight=[0.5,0.5]):
    for t in range(rounds):
        print(f"Round {t + 1}/{rounds}")

        m = max(int(fraction_clients * num_clients), 1)
        selected_clients = np.random.choice(num_clients, m, replace=False)

        client_models = []

        for client_idx in selected_clients:
            x_client, y_client = client_datasets[client_idx]

            with tf.device(device):
              if common_parameter:
                local_model = clone_model(global_model)
                local_model.set_weights(global_model.get_weights())
              else:
                local_model = create_model()
              local_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
                                  loss=loss,
                                  metrics=['accuracy'])
              local_model.fit(x_client, y_client, epochs=epochs, batch_size=batch_size, verbose=0)
              client_models.append(local_model)

        weights = [len(client_datasets[client_idx][0]) for client_idx in selected_clients]
        average_weight = [weight / sum(weights) for weight in weights]
        averaged_weights = average_model_parameters(client_models, average_weight)

        global_model.set_weights(averaged_weights)

    return global_model


7.Train your models without initializing the common parameters and measure the performance on the entire dataset.

In [24]:
#No initialization
global_model = create_model()

federated_model = federated_averaging(
    global_model=global_model,
    client_datasets=client_datasets,
    num_clients=2,
    rounds=10,
    epochs=10,
    batch_size=50,
    learning_rate=0.01,
    fraction_clients=1.0,
    common_parameter=False
)

federated_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
loss_global, acc_global = federated_model.evaluate(x_test[..., np.newaxis], y_test, verbose=0)

print(f"Global Model Accuracy: {acc_global:.2f}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Round 1/10
Round 2/10
Round 3/10
Round 4/10
Round 5/10
Round 6/10
Round 7/10
Round 8/10
Round 9/10
Round 10/10
Global Model Accuracy: 0.35


8.Train your models with the initialization of common parameters and verify that the performance is better.

In [22]:
#initialization
global_model = create_model()

federated_model = federated_averaging(
    global_model=global_model,
    client_datasets=client_datasets,
    num_clients=2,
    rounds=10,
    epochs=10,
    batch_size=50,
    learning_rate=0.01,
    fraction_clients=1.0
)

federated_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
loss_global, acc_global = federated_model.evaluate(x_test[..., np.newaxis], y_test, verbose=0)

print(f"Global Model Accuracy: {acc_global:.2f}")

Round 1/10
Round 2/10
Round 3/10
Round 4/10
Round 5/10
Round 6/10
Round 7/10
Round 8/10
Round 9/10
Round 10/10
Global Model Accuracy: 0.93


We can see that when the models are initialized with the same parameters (from global model), the final results are better.

9.Reduce the number of data points in each sub-batch. What is the minimum number of data points necessary for the final model to have acceptable performance? Repeat the study on CIFAR-10

In [49]:
client_datasets_50 = create_client_datasets(x_train, y_train, 50)
client_datasets_100 = create_client_datasets(x_train, y_train, 100)
client_datasets_150 = create_client_datasets(x_train, y_train, 150)
client_datasets_200 = create_client_datasets(x_train, y_train, 200)

all_clients_datasets = [client_datasets_50, client_datasets_100, client_datasets_150, client_datasets_200]
results = []

for i in range(len(all_clients_datasets)):
  cl_datasets = all_clients_datasets[i]
  global_model = create_model()

  federated_model = federated_averaging(
      global_model=global_model,
      client_datasets=cl_datasets,
      num_clients=2,
      rounds=7,
      epochs=10,
      batch_size=50,
      learning_rate=0.01,
      fraction_clients=1.0
  )

  federated_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  loss_global, acc_global = federated_model.evaluate(x_test[..., np.newaxis], y_test, verbose=0)
  results.append({'datasets_size': 50*(i+1),'loss': loss_global, 'accuracy': acc_global})

for result in results:
  print(result)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Round 1/7
Round 2/7
Round 3/7
Round 4/7
Round 5/7
Round 6/7
Round 7/7
Round 1/7
Round 2/7
Round 3/7
Round 4/7
Round 5/7
Round 6/7
Round 7/7
Round 1/7
Round 2/7
Round 3/7
Round 4/7
Round 5/7
Round 6/7
Round 7/7
Round 1/7
Round 2/7
Round 3/7
Round 4/7
Round 5/7
Round 6/7
Round 7/7
{'datasets_size': 50, 'loss': 2.224083185195923, 'accuracy': 0.21629999577999115}
{'datasets_size': 100, 'loss': 1.7315049171447754, 'accuracy': 0.4867999851703644}
{'datasets_size': 150, 'loss': 0.9667416214942932, 'accuracy': 0.7534999847412109}
{'datasets_size': 200, 'loss': 0.6597244143486023, 'accuracy': 0.7983999848365784}


Minimum minimum number of data points necessary for the final model to have acceptable performance is 150

## CYFAR-10 test

In [5]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

(x_train_cyphar, y_train_cyphar), (x_test_cyphar, y_test_cyphar) = cifar10.load_data()

x_train_cyphar = x_train_cyphar.astype('float32') / 255.0
x_test_cyphar = x_test_cyphar.astype('float32') / 255.0

y_train_cyphar = to_categorical(y_train_cyphar, 10)
y_test_cyphar = to_categorical(y_test_cyphar, 10)

print("Training samples:", x_train_cyphar.shape[0])
print("Test samples:", x_test_cyphar.shape[0])
print("Image shape:", x_train_cyphar.shape[1:])


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Training samples: 50000
Test samples: 10000
Image shape: (32, 32, 3)


In [13]:
def create_model():
  model = Sequential([
      Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
      MaxPooling2D((2, 2)),
      Conv2D(64, (3, 3), activation='relu'),
      MaxPooling2D((2, 2)),
      Flatten(),
      Dense(128, activation='relu'),
      Dense(10, activation='softmax')  # 10 classes for MNIST digits
  ])
  return model

In [15]:
client_datasets_cyphar_1500 = create_client_datasets(x_train_cyphar, y_train_cyphar, 1500)
client_datasets_cyphar_1750 = create_client_datasets(x_train_cyphar, y_train_cyphar, 1750)
client_datasets_cyphar_2000 = create_client_datasets(x_train_cyphar, y_train_cyphar, 2000)

all_clients_datasets_cyphar = [client_datasets_cyphar_1500, client_datasets_cyphar_1750, client_datasets_cyphar_2000]
sizes = [600, 1000, 1500]

results_cyphar = []
for i in range(len(all_clients_datasets_cyphar)):
  cl_datasets_cyphar = all_clients_datasets_cyphar[i]
  size = sizes[i]
  global_model = create_model()
  federated_model = federated_averaging(
      global_model=global_model,
      client_datasets=cl_datasets_cyphar,
      num_clients=2,
      rounds=7,
      epochs=10,
      batch_size=50,
      learning_rate=0.01,
      fraction_clients=1.0,
      loss = 'categorical_crossentropy'
  )

  federated_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
  loss_global, acc_global = federated_model.evaluate(x_test_cyphar[..., np.newaxis], y_test_cyphar, verbose=0)

  results_cyphar.append({'datasets_size': size, 'loss': loss_global, 'accuracy': acc_global})

for result in results_cyphar:
  print(result)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Round 1/7
Round 2/7
Round 3/7
Round 4/7
Round 5/7
Round 6/7
Round 7/7
Round 1/7
Round 2/7
Round 3/7
Round 4/7
Round 5/7
Round 6/7
Round 7/7
Round 1/7
Round 2/7
Round 3/7
Round 4/7
Round 5/7
Round 6/7
Round 7/7
{'datasets_size': 600, 'loss': 1.6058480739593506, 'accuracy': 0.4316999912261963}
{'datasets_size': 1000, 'loss': 1.5184733867645264, 'accuracy': 0.45820000767707825}
{'datasets_size': 1500, 'loss': 1.5298556089401245, 'accuracy': 0.46369999647140503}
